In [22]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import string

In [18]:
current_page = "https://tagalog.pinoydictionary.com/list/a/"
page = requests.get(current_page)
soup = BeautifulSoup(page.text, 'html')
subpages = soup.find_all("p", class_="pages")[0]
last_page = subpages.find_all("a")[-1].get("href")
num_subpages = int(re.search(r"\/[0-9]+\/$", last_page).group().replace("/", ""))

In [29]:
current_page = "https://tagalog.pinoydictionary.com/list/c/"
page = requests.get(current_page)
soup = BeautifulSoup(page.text, 'html')
subpages = soup.find_all("p", class_="pages")[0]
last_page = subpages.find_all("a")[-1].get("href")
last_page
#num_subpages = int(re.search(r"\/[0-9]+\/$", last_page).group().replace("/", ""))

'https://tagalog.pinoydictionary.com/list/z/'

In [19]:
pos_pattern = re.compile(r"\s?(\w+)\.(, \s?(\w+)\.)?")
definition_pattern = re.compile(r"(?:\s?(\w+)\.(, \s?(\w+)\.)?) (.*)")


In [36]:
data = []

In [20]:
for x in range(1, num_subpages + 1):
    current_page = f"https://tagalog.pinoydictionary.com/list/a/{'' if x == 1 else f'{x}/'}"
    page = requests.get(current_page)
    soup = BeautifulSoup(page.text, 'html')
    word_groups = soup.find_all("div", class_="word-group")
    
    for word in word_groups:
        
        tagalog_word = word.find("a").text.strip()
        raw_definition_text = word.find("div", class_="definition").text.strip()
        
        pos_match = pos_pattern.search(raw_definition_text)
        pos = pos_match.group(1) if pos_match else "Unknown"
        
        definition_match = definition_pattern.search(raw_definition_text)
        definition = definition_match.group(4) if definition_match else raw_definition_text
        
        data.append({"Tagalog": tagalog_word, "Part of Speech": pos, "Definition": definition})
        
df = pd.DataFrame(data)
df

,Tagalog,Part of Speech,Definition
0,aalug-alog,adj,shaky; rickety
1,aam,n,broth made from boiled rice
2,aanga-anga,adj,remaining agape for some time
3,aapa-apa,adj,groping around
4,aba,adj,poor; humble; subservient
...,...,...,...
1751,ayuno,n,fasting
1752,ayupin,v,"humiliate, aggrieve or do wrong to someone"
1753,ayusin,v,settle; put in order; arrange
1754,aywan,expr,"used like this: ""aywan ko"", ""I don't know"" [id..."


In [38]:
alpha = list(string.ascii_lowercase)

for letter in alpha:
    print(letter)
    if letter == "f" or letter == "q" or letter == "v":
        continue
    current_page = f"https://tagalog.pinoydictionary.com/list/{letter}/"
    page = requests.get(current_page)
    soup = BeautifulSoup(page.text, 'html')
    subpages = soup.find_all("p", class_="pages")[0]
    last_page = subpages.find_all("a")[-1].get("href")
    if letter == "c" or letter == "j" or letter == "x" or letter == "z":
        num_subpages = 1
    else:
        num_subpages = int(re.search(r"\/[0-9]+\/$", last_page).group().replace("/", ""))
    
    for x in range(1, num_subpages + 1):
        current_page = f"https://tagalog.pinoydictionary.com/list/{letter}/{'' if x == 1 else f'{x}/'}"
        page = requests.get(current_page)
        soup = BeautifulSoup(page.text, 'html')
        word_groups = soup.find_all("div", class_="word-group")
    
        for word in word_groups:
        
            tagalog_word = word.find("a").text.strip()
            raw_definition_text = word.find("div", class_="definition").text.strip()
        
            pos_match = pos_pattern.search(raw_definition_text)
            pos = pos_match.group(1) if pos_match else "Unknown"
        
            definition_match = definition_pattern.search(raw_definition_text)
            definition = definition_match.group(4) if definition_match else raw_definition_text
        
            data.append({"Tagalog": tagalog_word, "Part of Speech": pos, "Definition": definition})
        
df = pd.DataFrame(data)
df

a
b
c
d
e
f
g
h
i
j
k
l
m
n
o
p
q
r
s
t
u
v
w
x
y
z


,Tagalog,Part of Speech,Definition
0,aalug-alog,adj,shaky; rickety
1,aam,n,broth made from boiled rice
2,aanga-anga,adj,remaining agape for some time
3,aapa-apa,adj,groping around
4,aba,adj,poor; humble; subservient
...,...,...,...
49207,yamutin,v,1. annoy; 2. displease; 3. pester
49208,yamutmot,n,1. rubbish; refuse; 2. loose frayed ends of th...
49209,yamuyam,n,scrapings; filings
49210,zink,n,zinc (Zn) (metal)


In [39]:
df.to_csv("tagalog_vocab.csv", index=False)